# 准备config配置文件

同济子豪兄 2023-2-13 6-11

## 进入MMSegmentation主目录

In [1]:
import os
os.chdir('../../mmsegmentation')

In [2]:
os.getcwd()

'/home/cine/Documents/GitHub/mmsegmentation'

## 导入工具包

In [3]:
import numpy as np
from PIL import Image

import os.path as osp
from tqdm import tqdm

import mmcv
import mmengine
import matplotlib.pyplot as plt
%matplotlib inline

## 定义数据集类（各类别名称及配色）

In [ ]:
# !rm -rf mmseg/datasets/DubaiDataset.py # 删除原有文件
# !wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/Dubai/DubaiDataset.py -P mmseg/datasets


## 注册数据集类

In [ ]:
# !rm -rf mmseg/datasets/__init__.py # 删除原有文件
# !wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/Dubai/__init__.py -P mmseg/datasets


## 定义训练及测试pipeline

In [ ]:
# !rm -rf configs/_base_/datasets/DubaiDataset_pipeline.py
# !wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/Dubai/DubaiDataset_pipeline.py -P configs/_base_/datasets


## 下载模型config配置文件

In [ ]:
# !rm -rf configs/pspnet/pspnet_r50-d8_4xb2-40k_DubaiDataset.py # 删除原有文件
# !wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20230130-mmseg/Dubai/pspnet_r50-d8_4xb2-40k_DubaiDataset.py -P configs/pspnet 


## 载入config配置文件

In [4]:
from mmengine import Config
cfg = Config.fromfile('./configs/pspnet/pspnet_r50-d8_4xb2-40k_DubaiDataset.py')

## 修改config配置文件

In [5]:
cfg.norm_cfg = dict(type='BN', requires_grad=True) # 只使用GPU时，BN取代SyncBN
cfg.crop_size = (256, 256)
cfg.model.data_preprocessor.size = cfg.crop_size
cfg.model.backbone.norm_cfg = cfg.norm_cfg
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# modify num classes of the model in decode/auxiliary head

# 模型 decode/auxiliary 输出头，指定为类别个数
cfg.model.decode_head.num_classes = 6
cfg.model.auxiliary_head.num_classes = 6

cfg.train_dataloader.batch_size = 12

cfg.test_dataloader = cfg.val_dataloader

# 结果保存目录
cfg.work_dir = './work_dirs/DubaiDataset'

# 训练迭代次数
cfg.train_cfg.max_iters = 8000
# 评估模型间隔
cfg.train_cfg.val_interval = 400
# 日志记录间隔
cfg.default_hooks.logger.interval = 100
# 模型权重保存间隔
cfg.default_hooks.checkpoint.interval = 1000

# 随机数种子
cfg['randomness'] = dict(seed=0)

In [6]:
# optimizer = dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0005)
cfg['randomness'] = dict(seed=42)
cfg.optimizer.lr=0.001
cfg['optimizer']

{'type': 'SGD', 'lr': 0.001, 'momentum': 0.9, 'weight_decay': 0.0005}

In [7]:
cfg.optim_wrapper

{'type': 'OptimWrapper',
 'optimizer': {'type': 'SGD',
  'lr': 0.01,
  'momentum': 0.9,
  'weight_decay': 0.0005},
 'clip_grad': None}

In [8]:
cfg.optim_wrapper.optimizer = cfg.optimizer
cfg.optim_wrapper

{'type': 'OptimWrapper',
 'optimizer': {'type': 'SGD',
  'lr': 0.001,
  'momentum': 0.9,
  'weight_decay': 0.0005},
 'clip_grad': None}

In [12]:
cfg.train_pipeline

[{'type': 'LoadImageFromFile'},
 {'type': 'LoadAnnotations'},
 {'type': 'RandomResize',
  'scale': (2048, 1024),
  'ratio_range': (0.5, 2.0),
  'keep_ratio': True},
 {'type': 'RandomCrop', 'crop_size': (64, 64), 'cat_max_ratio': 0.75},
 {'type': 'RandomFlip', 'prob': 0.5},
 {'type': 'PhotoMetricDistortion'},
 {'type': 'PackSegInputs'}]

In [13]:
cfg.train_pipeline[3]

{'type': 'RandomCrop', 'crop_size': (64, 64), 'cat_max_ratio': 0.75}

In [14]:
cfg.train_pipeline[3].crop_size

(64, 64)

In [15]:
cfg.train_pipeline[3].crop_size = cfg.crop_size
cfg.train_pipeline[3].crop_size

(256, 256)

In [24]:
cfg.train_pipeline

[{'type': 'LoadImageFromFile'},
 {'type': 'LoadAnnotations'},
 {'type': 'RandomResize',
  'scale': (2048, 1024),
  'ratio_range': (0.5, 2.0),
  'keep_ratio': True},
 {'type': 'RandomCrop', 'crop_size': (256, 256), 'cat_max_ratio': 0.75},
 {'type': 'RandomFlip', 'prob': 0.5},
 {'type': 'PhotoMetricDistortion'},
 {'type': 'PackSegInputs'}]

In [20]:
cfg.train_dataloader

{'batch_size': 12,
 'num_workers': 2,
 'persistent_workers': True,
 'sampler': {'type': 'InfiniteSampler', 'shuffle': True},
 'dataset': {'type': 'DubaiDataset',
  'data_root': 'Dubai-dataset/',
  'data_prefix': {'img_path': 'img_dir/train',
   'seg_map_path': 'ann_dir/train'},
  'pipeline': [{'type': 'LoadImageFromFile'},
   {'type': 'LoadAnnotations'},
   {'type': 'RandomResize',
    'scale': (2048, 1024),
    'ratio_range': (0.5, 2.0),
    'keep_ratio': True},
   {'type': 'RandomCrop', 'crop_size': (64, 64), 'cat_max_ratio': 0.75},
   {'type': 'RandomFlip', 'prob': 0.5},
   {'type': 'PhotoMetricDistortion'},
   {'type': 'PackSegInputs'}]}}

In [26]:
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
cfg.train_dataloader.dataset.pipeline

[{'type': 'LoadImageFromFile'},
 {'type': 'LoadAnnotations'},
 {'type': 'RandomResize',
  'scale': (2048, 1024),
  'ratio_range': (0.5, 2.0),
  'keep_ratio': True},
 {'type': 'RandomCrop', 'crop_size': (256, 256), 'cat_max_ratio': 0.75},
 {'type': 'RandomFlip', 'prob': 0.5},
 {'type': 'PhotoMetricDistortion'},
 {'type': 'PackSegInputs'}]

## 查看完整config配置文件

In [27]:
print(cfg.pretty_text)

norm_cfg = dict(type='BN', requires_grad=True)
data_preprocessor = dict(
    type='SegDataPreProcessor',
    mean=[123.675, 116.28, 103.53],
    std=[58.395, 57.12, 57.375],
    bgr_to_rgb=True,
    pad_val=0,
    seg_pad_val=255,
    size=(64, 64))
model = dict(
    type='EncoderDecoder',
    data_preprocessor=dict(
        type='SegDataPreProcessor',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        bgr_to_rgb=True,
        pad_val=0,
        seg_pad_val=255,
        size=(256, 256)),
    pretrained='open-mmlab://resnet50_v1c',
    backbone=dict(
        type='ResNetV1c',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        dilations=(1, 1, 2, 4),
        strides=(1, 2, 1, 1),
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
        contract_dilation=True),
    decode_head=dict(
        type='PSPHead',
        in_channels=2048,
        in_index=3,
        channels=51

## 保存config配置文件

In [28]:
cfg.dump('pspnet-DubaiDataset_20230615-3_crop_size.py')